### Import Datasets

In [ ]:
import numpy as np
from glob import glob

human_path = "data/lfw/*/*"
dog_path = "data/dog_images/*/*/*"

human_files = np.array(glob( human_path ))
dog_files = np.array(glob( dog_path ))

print('%d Total human images.' % len(human_files))
print('%d Total dog images.' % len(dog_files))

In [ ]:
print(type(glob("data/lfw/*/*")))
print(human_files[0:5])
print(dog_files[0:5])

### Detect Humans
[Haar feature-based cascade classifiers](http://docs.opencv.org/trunk/d7/d8b/tutorial_py_face_detection.html)

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

img = cv2.imread(human_files[15]) # <class 'numpy.ndarray'>
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert image to grayscale
faces = face_cascade.detectMultiScale(gray) # numpy array of detected faces in grayscale

print('Number of faces detected: {}'.format(len(faces)))

for (x, y, w, h) in faces:
    cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2) # (image, start point, end point, color, pixel thickness)

cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(cv_rgb)
plt.show()

### Human Face Detector
Function to return True if human face is detected, else return False

In [ ]:
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

print(face_detector(human_files[10]))
print(face_detector(dog_files[10]))

#### Assess the Human Face Detector

In [ ]:
from tqdm import tqdm
import sys

human_files_short = human_files[:50]
dog_files_short = dog_files[:50]

human_face_counter = 0
dog_face_counter = 0

dog_and_human = []

def haar_tester():

    for i in tqdm(range(len(human_files_short))):
        human_face = face_detector(human_files_short[i])
        dog_face = face_detector(dog_files_short[i])
        human_face_counter += human_face
        dog_face_counter += dog_face

        if dog_face == True:
            dog_and_human.append(i)
    
    print("Human Imgs Face Detection %age: \t{}".format(human_face_counter / 100))
    print("Dog Imgs Face Detection %age: \t{}".format(dog_face_counter / 100))
    print("Ten entries from dog_and_human: {}".format(dog_and_human[:10]))

In [ ]:
img = cv2.imread(dog_files_short[15])
plt.imshow(img)
plt.show()

In [ ]:
import os
os.listdir('haarcascades/')

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt2.xml')

human_face_counter = 0
dog_face_counter = 0
dog_and_human = []

haar_tester()

In [ ]:
img = cv2.imread(dog_files_short[1])
plt.imshow(img)
plt.show()

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

human_face_counter = 0
dog_face_counter = 0
dog_and_human = []

for i in tqdm(range(len(human_files_short))):
    human_face = face_detector(human_files_short[i])
    dog_face = face_detector(dog_files_short[i])
    human_face_counter += human_face
    dog_face_counter += dog_face
    
    if dog_face == True:
        dog_and_human.append(i)
    
print("Human Imgs Face Detection %age: \t{}".format(human_face_counter / 50))
print("Dog Imgs Face Detection %age: \t{}".format(dog_face_counter / 50))
print("Ten entries from dog_and_human: {}".format(dog_and_human[:10]))

### Detect Dogs Using Pre-trained VGG16, Standard labels

In [ ]:
import torch
import torchvision.models as models

VGG16 = models.vgg16(pretrained=True)

if torch.cuda.is_available():
    VGG16 = VGG16.cuda()
    print("VGG16 has been moved to cuda device.".format(VGG16))

for param in VGG16.features.parameters():
    param.requires_grad = False
    
VGG16

In [ ]:
from matplotlib.pyplot import imshow
from PIL import Image
import torchvision.transforms as transforms

def VGG16_predict(img_path):

    img = Image.open(img_path)

    transform = transforms.Compose([transforms.Resize((256,256)),
                                    transforms.CenterCrop(224),
                                    transforms.RandomRotation(35),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.47,0.47,0.47),
                                                        (0.47,0.47,0.47))])

    image_tensor = transform(img)
    image_tensor = image_tensor.unsqueeze(0)

    if torch.cuda.is_available():
        image_tensor = image_tensor.cuda()

    prediction = VGG16(image_tensor)
    prediction = prediction.cpu()

    predicted_class_index = prediction.data.numpy().argmax()

    return predicted_class_index

In [ ]:
from PIL import Image

def dog_detector(img_path): 
    #Return true if dog is detected
    #Index between 151 and 268 inclusive

    predicted_class_index = VGG16_predict(img_path)

    if predicted_class_index >= 151 and predicted_class_index <= 268:
        return True
    else:
        return False

In [ ]:
test_cases = 50
dog_counter = []

for i in range(test_cases):
    dog_detected = dog_detector(dog_files[i])
    if dog_detected == True:
        dog_counter.append(1)
    else:
        dog_counter.append(0)

result = np.asarray(dog_counter)
print(result.mean(), " Accuracy for 50 test cases")

test_cases = 500
dog_counter = []

for i in range(test_cases):
    dog_detected = dog_detector(dog_files[i])
    if dog_detected == True:
        dog_counter.append(1)
    else:
        dog_counter.append(0)

result = np.asarray(dog_counter)
print(result.mean(), "Accuracy for 500 test cases")

test_cases = 5000
dog_counter = []

for i in range(test_cases):
    dog_detected = dog_detector(dog_files[i])
    if dog_detected == True:
        dog_counter.append(1)
    else:
        dog_counter.append(0)

result = np.asarray(dog_counter)
print(result.mean(), "Accuracy for 5000 test cases")

### Final data pre-processing

In [5]:
# Function for generating labels
import pandas as pd
import numpy as np
from glob import glob

def gen_labels(list_of_paths):
    df = pd.DataFrame(list_of_paths)
    df.rename(columns={0:"file_path"}, inplace=True)
    df['label'] = None
    
    for i in range(len(df)):
        if 'lfw' in df.loc[i,'file_path']:
            df.loc[i,'label'] = 0
        elif 'dog' in df.loc[i,'file_path']:
            myString = df.loc[i,'file_path']
            if myString.find('\\') > 0:
                cut1, cut2, cut3, cut4 = myString.split("\\")
                dogLabel, discard = cut3.split('.')
                df.loc[i,'label'] = int(dogLabel)
    
    return df

def fill_human_frame(source_df, target_index):
    new_df = pd.DataFrame(columns={'file_path','label'})
    counter = 0
    
    for i in target_index:
        a, b = source_df.loc[i, ['file_path', 'label']]
        new_df.loc[counter,['file_path','label']] = a, b
        counter += 1
    
    return new_df

def fill_dog_frame(source_df, train_type=None):
    new_df = pd.DataFrame(columns={'file_path', 'label'})
    counter = 0
    
    for i in range(len(source_df)):
        if train_type in source_df.loc[i,'file_path']:
            new_df.loc[i,['file_path','label']] = source_df.loc[i]
            counter += 0
    
    new_df.reset_index(inplace=True)
    new_df.drop(['index'], axis=1, inplace=True)
    
    return new_df

def shuffle_reset_index(df):
    new_df = df
    new_df = new_df.sample(frac=1)
    new_df = new_df.reset_index()
    new_df = new_df.drop(['index'], axis=1)
    
    return new_df

In [6]:
human_paths_all = gen_labels(glob('data/lfw/*/*'))
dog_paths_all = gen_labels(glob('data/dog_images/*/*/*'))

In [7]:
# Create shuffled indices for human images b/c they were not pre-specified as train/valid/test
human_indices = list(range(len(human_paths_all)))
np.random.shuffle(human_indices)
human_train_idx = human_indices[:int(len(human_indices) * 0.8)]
x = human_indices[-int(len(human_indices) * 0.2):]
human_valid_idx = x[:int(len(x) * 0.5)]
human_test_idx = x[-int(len(x) * 0.5):]

human_data_train = fill_human_frame(human_paths_all, human_train_idx)
human_data_test = fill_human_frame(human_paths_all, human_valid_idx)
human_data_valid = fill_human_frame(human_paths_all, human_test_idx)

dog_data_train = fill_dog_frame(dog_paths_all, train_type="train")
dog_data_valid = fill_dog_frame(dog_paths_all, train_type="valid")
dog_data_test = fill_dog_frame(dog_paths_all, train_type="test")

In [8]:
# Concatenate human and dog data, shuffle again and reset index.

train_data_df = pd.concat([dog_data_train, human_data_train])
test_data_df = pd.concat([dog_data_test, human_data_test])
valid_data_df = pd.concat([dog_data_valid, human_data_valid])

train_data_df = shuffle_reset_index(train_data_df)
test_data_df = shuffle_reset_index(test_data_df)
valid_data_df = shuffle_reset_index(valid_data_df)

train_data_df.head()

,file_path,label
0,data/lfw\Luiz_Inacio_Lula_da_Silva\Luiz_Inacio...,0
1,data/lfw\John_Goold\John_Goold_0001.jpg,0
2,data/lfw\George_Robertson\George_Robertson_001...,0
3,data/lfw\Gerhard_Schroeder\Gerhard_Schroeder_0...,0
4,data/lfw\Hanns_Schumacher\Hanns_Schumacher_000...,0


### Data Exploration; Re-map our dog labels with VGG16 imagenet1000 labels
https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a#file-imagenet1000_clsidx_to_labels-txt

In [9]:
import numpy as np
from glob import glob

raw_labels = open('data/imagenet1000_clsidx_to_labels.txt', 'r')
raw_labels = list(raw_labels)
reference_dog_labels = raw_labels[151:269]
print(len(reference_dog_labels))
reference_dog_labels[0:5]

for i in range(len(reference_dog_labels)):
    reference_dog_labels[i] = reference_dog_labels[i].replace(',\n','')
    if reference_dog_labels[i][0] == " ":
        reference_dog_labels[i] = reference_dog_labels[i][1:]

ref_dog_dict = {}
        
for i in reference_dog_labels:
    i = i.split(": ")
    i[1] = i[1].replace("'", "")
    ref_dog_dict[i[0]] = i[1]

118


In [ ]:
dog_path_train_check = np.array(glob('data/dog_images/train/*'))
dog_path_test_check = np.array(glob('data/dog_images/test/*'))
dog_path_valid_check = np.array(glob('data/dog_images/valid/*'))

# Function to help clean up the glob path lists
def label_list_maker(glob_list):
    new_list = []
    for item in glob_list:
        if item.find('\\') != 0:
            cut_off = item.find('\\') + 1
            item = item[cut_off:]
        new_list.append(item)  
    return new_list

# Concatenate all dog glob paths 
# This will help check for dog breeds which may be included in one data set, but not the other
concat =  label_list_maker(dog_path_train_check) + label_list_maker(dog_path_test_check) + label_list_maker(dog_path_valid_check)
concat = list(dict.fromkeys(concat)) # Make a set

# Instantiate empty dictionary to serve as our dog labels dict
# Later we will check to see how this maps to the Imagenet 1000 categories
myDog_labels_dict = {}

for i in concat:
    key = str(i.split(".")[0])
    value = str(i.split(".")[1])
    myDog_labels_dict[key] = value

In [ ]:
print(len(ref_dog_dict))
print("myDog_labels_dict['01']: %s" % ref_dog_dict['001'])
print("myDog_labels_dict['118']: %s" % ref_dog_dict['118'])

# Our dog labels dictionary complete:
print(len(myDog_labels_dict))
print("myDog_labels_dict['001']: %s" % myDog_labels_dict['001'])
print("myDog_labels_dict['133']: %s" % myDog_labels_dict['133'])

### Data Observation
118: Number of dog breed categories in Imagenet 1000<br />
133: Number of dog breed categories in our dataset<br />
This is likely the reason why VGG16 only achieved 88% accuracy when we ran our dataset on it.

### Create Dataloader. Data Input: image_path, label

In [10]:
import numpy as np
import pandas as pd
import os
import torchvision.transforms as transforms
from PIL import Image
from glob import glob
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    
    def __init__(self, img_paths, labels=None, transform=None):
        self.img_paths = img_paths
        self.to_image = []
        self.len = len(self.img_paths)
        self.transform = transform
        
        if labels is not None:
            self.labels = labels
        else:
            self.labels = None
        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        self.img_paths_temp = Image.open(self.img_paths[idx])
        
        if self.transform is not None:
            self.img_paths_temp = transform(self.img_paths_temp)
        
        X = self.img_paths_temp
        
        if self.labels is not None:
            y = self.labels[idx]
        else:
            y = 'No Label Found!'
        
        return (X, y)

In [11]:
from torch.utils.data import DataLoader

batch_size = 16
num_workers = 0
transform = transforms.Compose([transforms.Resize(250),
                             transforms.CenterCrop(224),
                             transforms.RandomRotation(30),
                             transforms.ToTensor()])

train_data = CustomDataset(train_data_df['file_path'], train_data_df['label'], transform=transform)
valid_data = CustomDataset(valid_data_df['file_path'], valid_data_df['label'], transform=transform)
test_data = CustomDataset(test_data_df['file_path'], test_data_df['label'], transform=transform)

train_loader = DataLoader(train_data, batch_size=batch_size, num_workers=0)
valid_loader = DataLoader(valid_data, batch_size=batch_size, num_workers=0)
test_loader = DataLoader(test_data, batch_size=batch_size, num_workers=0)

### Define Network Architecture

In [15]:
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self, ):
        super(Network, self).__init__()
        self.conv1_1 = nn.Conv2d(3, 12, 3, stride=1, padding=1)
        self.conv1_2 = nn.Conv2d(12, 12, 3, stride=1, padding=1)
        self.conv1_3 = nn.Conv2d(12, 24, 3, stride=1, padding=1)
        self.conv2_1 = nn.Conv2d(24, 48, 3, stride=1, padding=1)
        #self.conv2_2 = nn.Conv2d(48, 48, 3, stride=1, padding=1)
        self.conv2_3 = nn.Conv2d(48, 96, 3, stride=1, padding=1)
        self.conv3_1 = nn.Conv2d(96, 128, 3, stride=1, padding=1)
        self.conv3_2 = nn.Conv2d(128, 128, 3, stride=1, padding=1)
        #self.conv3_3 = nn.Conv2d(128, 128, 3, stride=1, padding=1)
        #self.conv3_4 = nn.Conv2d(128, 256, 3, stride=1, padding=1)
        self.conv4_1 = nn.Conv2d(128, 128, 3, stride=1, padding=1)
        #self.conv4_2 = nn.Conv2d(256, 256, 3, stride=1, padding=1)
        #self.conv4_3 = nn.Conv2d(256, 512, 3, stride=1, padding=1)
        
        self.maxPool = nn.MaxPool2d(2, 2, 1)
        self.dropout = nn.Dropout(p=0.50)
        
        self.fc_1 = nn.Linear(28800,2880)
        self.fc_2 = nn.Linear(2880,512)
        self.fc_3 = nn.Linear(512,134)
        
    def forward(self, x):
        #print("Before conv1_1: {}".format(x.shape))
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = F.relu(self.conv1_3(x))
        x = self.maxPool(x)
        x = F.relu(self.conv2_1(x))
        #x = F.relu(self.conv2_2(x))
        x = F.relu(self.conv2_3(x))
        x = self.maxPool(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        #x = F.relu(self.conv3_3(x))
        #x = F.relu(self.conv3_4(x))
        x = self.maxPool(x)
        x = F.relu(self.conv4_1(x))
        #x = F.relu(self.conv4_2(x))
        #x = F.relu(self.conv4_3(x))
        x = self.maxPool(x)
        
        x = x.view(x.shape[0], -1)
        #print("After view, before relu: {}".format(x.shape))
        x = F.relu(self.fc_1(x))
        x = self.dropout(x)
        x = F.relu(self.fc_2(x))
        x = self.dropout(x)
        x = self.fc_3(x)
        return x
    
model = Network()
model

Network(
  (conv1_1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_3): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_1): Conv2d(24, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_3): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_1): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxPool): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_1): Linear(in_features=28800, out_features=2880, bias=True)
  (fc_2): Linear(in_features=2880, out_features=512, bias=True)
  (fc_3): Linear(in_features=512, out_features=134, bias=True)
)

In [16]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = "cuda"
model.cuda()

Network(
  (conv1_1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_3): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_1): Conv2d(24, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_3): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_1): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxPool): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_1): Linear(in_features=28800, out_features=2880, bias=True)
  (fc_2): Linear(in_features=2880, out_features=512, bias=True)
  (fc_3): Linear(in_features=512, out_features=134, bias=True)
)

In [17]:
import torch
from torchvision.transforms import transforms
from PIL import ImageFile

torch.cuda.empty_cache()
ImageFile.LOAD_TRUNCATED_IMAGES = True
n_epochs = 10

valid_loss_min = np.Inf

for epoch in range(1, n_epochs+1):

    train_loss = 0.0
    valid_loss = 0.0
    
    model.train()
    modulus = 0
    
    for data, target in train_loader:
        
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
          
    # Validate model
    
    model.eval()
    
    for data, target in valid_loader:

        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()

        output = model(data)

        loss = criterion(output, target)
        
        valid_loss += loss.item()*data.size(0)
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_murpher_newfcs.pt')
        valid_loss_min = valid_loss

trainloss: 78.49901580810547
trainloss: 148.96137237548828
trainloss: 308.73247146606445
trainloss: 568.5750999450684
trainloss: 1204.1608428955078
trainloss: 2313.099245071411
Epoch: 1 	Training Loss: 2.703021 	Validation Loss: 2.561246
Validation loss decreased (inf --> 2.561246).  Saving model ...
trainloss: 49.93525314331055
trainloss: 84.97062683105469
trainloss: 217.46641540527344
trainloss: 402.15576553344727
trainloss: 1011.2337760925293
trainloss: 1996.0555248260498
Epoch: 2 	Training Loss: 2.607522 	Validation Loss: 2.558068
Validation loss decreased (2.561246 --> 2.558068).  Saving model ...
trainloss: 50.38090896606445
trainloss: 84.79438781738281
trainloss: 212.38259887695312
trainloss: 397.1840057373047
trainloss: 993.7785339355469
trainloss: 1947.4501132965088
Epoch: 3 	Training Loss: 2.591977 	Validation Loss: 2.557171
Validation loss decreased (2.558068 --> 2.557171).  Saving model ...
trainloss: 49.25724792480469
trainloss: 84.53406524658203
trainloss: 211.97245788574

RuntimeError: CUDA out of memory. Tried to allocate 318.00 MiB (GPU 0; 8.00 GiB total capacity; 5.30 GiB already allocated; 101.55 MiB free; 6.17 GiB reserved in total by PyTorch)

In [18]:
for data, target in test_loader:

    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()

    output = model(data)
    loss = criterion(output, target)
    print(loss)

    valid_loss += loss.item()*data.size(0)

RuntimeError: CUDA out of memory. Tried to allocate 76.00 MiB (GPU 0; 8.00 GiB total capacity; 5.80 GiB already allocated; 25.55 MiB free; 6.24 GiB reserved in total by PyTorch)

In [ ]:
fds